### Ejercicio SQL


**Diego Alejandro Alzate Rodríguez**

- Construir consultas sobre la base de datos del ICFES 2024

In [3]:
import os
os.getcwd()
path = 'C:\\Users\\djdie\\Documents\\GitHub\\Diplomado_Gestion_Datos\\Fundamentos de Python\\data'
os.chdir(path)

In [4]:
import pandas as pd
import sqlite3
import os


# Ruta del archivo CSV
parquet_file = path + "\\SB11_20232.parquet"  # Parquet file

# Nombre de la tabla en SQLite
table_name = 'data_icfes'

# Crear conexión SQLite (archivo físico o en memoria)
conn = sqlite3.connect('data.db')  # Guardará la base de datos como 'data.db'

# Leer el archivo CSV como un DataFrame
df = pd.read_parquet(parquet_file)

# Cargar el DataFrame en SQLite
df.to_sql(table_name, conn, index=False, if_exists='replace')  # Reemplaza si ya existe

# Verificar que se cargó correctamente con una consulta
pd.options.display.max_columns = 100
query = "SELECT * FROM data_icfes LIMIT 5"
result = pd.read_sql_query(query, conn)
result.columns


Index(['ESTU_TIPODOCUMENTO', 'ESTU_NACIONALIDAD', 'ESTU_GENERO',
       'ESTU_FECHANACIMIENTO', 'PERIODO', 'ESTU_CONSECUTIVO',
       'ESTU_ESTUDIANTE', 'ESTU_PAIS_RESIDE', 'ESTU_DEPTO_RESIDE',
       'ESTU_COD_RESIDE_DEPTO', 'ESTU_MCPIO_RESIDE', 'ESTU_COD_RESIDE_MCPIO',
       'ESTU_PRESENTACIONSABADO', 'ESTU_LENGUANATIVA', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'FAMI_TRABAJOLABORPADRE',
       'FAMI_TRABAJOLABORMADRE', 'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEHORNOMICROOGAS',
       'FAMI_TIENESERVICIOTV', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENEMOTOCICLETA',
       'FAMI_TIENECONSOLAVIDEOJUEGOS', 'FAMI_NUMLIBROS',
       'FAMI_COMELECHEDERIVADOS', 'FAMI_COMECARNEPESCADOHUEVO',
       'FAMI_COMECEREALFRUTOSLEGUMBRE', 'FAMI_SITUACIONECONOMICA',
       'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_DEDICACIONINTERNET',
       'ESTU_HORASSEMANATRABAJA', 'ESTU_TIPOREMUNER

In [5]:
# Conteo de la cantidad de estudiantes que presentaron la prueba por departamento
query = '''SELECT 
    ESTU_DEPTO_PRESENTACION, 
    COUNT(*) AS Total_Estudiantes
FROM data_icfes
GROUP BY ESTU_DEPTO_PRESENTACION
ORDER BY Total_Estudiantes DESC;
'''
result = pd.read_sql_query(query, conn)
result

,ESTU_DEPTO_PRESENTACION,Total_Estudiantes
0,BOGOTÁ,78152
1,ANTIOQUIA,73834
2,VALLE,38837
3,CUNDINAMARCA,36720
4,ATLANTICO,32606
5,SANTANDER,26698
6,BOLIVAR,26190
7,CORDOBA,22323
8,NORTE SANTANDER,18176
9,MAGDALENA,17690


In [6]:
# Porcentaje de estudiantes calendario A
query = '''SELECT 
    ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM data_icfes)), 2) AS Porcentaje_Estudiantes_Calendario_A
FROM data_icfes
WHERE COLE_CALENDARIO = 'A';
'''
result = pd.read_sql_query(query, conn)
result

,Porcentaje_Estudiantes_Calendario_A
0,99.59


In [7]:
# Promedio del puntaje global y mediana
query = '''WITH OrderedScores AS (
    SELECT 
        PUNT_GLOBAL, 
        ROW_NUMBER() OVER (ORDER BY PUNT_GLOBAL) AS RowNum,
        COUNT(*) OVER () AS Total
    FROM data_icfes
),
MedianValue AS (
    SELECT 
        CASE 
            WHEN Total % 2 = 0 THEN 
                (SELECT AVG(PUNT_GLOBAL) 
                 FROM OrderedScores 
                 WHERE RowNum IN (Total / 2, Total / 2 + 1))
            ELSE 
                (SELECT PUNT_GLOBAL 
                 FROM OrderedScores 
                 WHERE RowNum = (Total + 1) / 2)
        END AS Mediana_Puntaje_Global
    FROM OrderedScores
    LIMIT 1
)
SELECT 
    AVG(PUNT_GLOBAL) AS Promedio_Puntaje_Global,
    (SELECT Mediana_Puntaje_Global FROM MedianValue) AS Mediana_Puntaje_Global
FROM data_icfes;
'''
result = pd.read_sql_query(query, conn)
result

,Promedio_Puntaje_Global,Mediana_Puntaje_Global
0,252.583392,249


In [8]:
# Recategorización del trabajo madre y promedio de puntaje en matemáticas
query = '''SELECT 
    CASE 
        WHEN FAMI_TRABAJOLABORMADRE IN ('No aplica', 'No sabe') OR FAMI_TRABAJOLABORMADRE IS NULL THEN 'No aplica'
        WHEN FAMI_TRABAJOLABORMADRE IN ('Trabaja en el hogar, no trabaja o estudia', 'Pensionado') THEN 'Sin empleo'
        WHEN FAMI_TRABAJOLABORMADRE IN (
            'Es agricultor, pesquero o jornalero',
            'Es operario de máquinas o conduce vehículos (taxita, chofer)',
            'Trabaja como personal de limpieza, mantenimiento, seguridad o construcción',
            'Trabaja por cuenta propia (por ejemplo plomero, electricista)'
        ) THEN 'Agricultura y trabajos manuales'
        WHEN FAMI_TRABAJOLABORMADRE IN (
            'Es dueño de un negocio pequeño (tiene pocos empleados o no tiene, por ejemplo tienda, papelería, etc',
            'Es dueño de un negocio grande, tiene un cargo de nivel directivo o gerencial'
        ) THEN 'Emprendedores y negocios'
        WHEN FAMI_TRABAJOLABORMADRE IN (
            'Trabaja como profesional (por ejemplo médico, abogado, ingeniero)',
            'Tiene un trabajo de tipo auxiliar administrativo (por ejemplo, secretario o asistente)'
        ) THEN 'Profesionales y administrativos'
        WHEN FAMI_TRABAJOLABORMADRE = 'Es vendedor o trabaja en atención al público' THEN 'Ventas y atención al cliente'
        ELSE 'Otros'
    END AS TrabajoLabor_Madre,
    ESTU_DEPTO_PRESENTACION,
    AVG(PUNT_MATEMATICAS) AS Promedio_Puntaje_Matematicas
FROM data_icfes
GROUP BY TrabajoLabor_Madre, ESTU_DEPTO_PRESENTACION
ORDER BY TrabajoLabor_Madre, ESTU_DEPTO_PRESENTACION;
'''
result = pd.read_sql_query(query, conn)
result

,TrabajoLabor_Madre,ESTU_DEPTO_PRESENTACION,Promedio_Puntaje_Matematicas
0,Agricultura y trabajos manuales,AMAZONAS,40.494444
1,Agricultura y trabajos manuales,ANTIOQUIA,48.331069
2,Agricultura y trabajos manuales,ARAUCA,48.910798
3,Agricultura y trabajos manuales,ATLANTICO,48.229930
4,Agricultura y trabajos manuales,BOGOTÁ,52.074920
...,...,...,...
193,Ventas y atención al cliente,SUCRE,48.116822
194,Ventas y atención al cliente,TOLIMA,50.604322
195,Ventas y atención al cliente,VALLE,50.189124
196,Ventas y atención al cliente,VAUPES,40.400000


In [9]:
# Distribución del genero de los estudiantes
query1 = '''
SELECT ESTU_GENERO, COUNT(*) AS Total_Estudiantes
FROM data_icfes
GROUP BY ESTU_GENERO;
'''
result = pd.read_sql_query(query1, conn)
result

,ESTU_GENERO,Total_Estudiantes
0,None,7
1,F,296927
2,M,254215


In [10]:
# Promedio mejores 10 puntajes globales por departamento de residencia
query2 = ''' 
SELECT ESTU_DEPTO_RESIDE, AVG(PUNT_GLOBAL) AS Promedio_Puntaje_Global
FROM data_icfes
GROUP BY ESTU_DEPTO_RESIDE
ORDER BY Promedio_Puntaje_Global DESC
LIMIT(10);

'''

result = pd.read_sql_query(query2, conn)
result

,ESTU_DEPTO_RESIDE,Promedio_Puntaje_Global
0,BOGOTÁ,271.339022
1,BOYACA,270.637017
2,SANTANDER,269.337855
3,QUINDIO,266.292342
4,CUNDINAMARCA,262.640345
5,NORTE SANTANDER,261.887976
6,RISARALDA,256.681414
7,HUILA,256.087842
8,CALDAS,256.052155
9,META,255.967349


In [11]:
# Relación del nivel socioeconómico y puntajes globales
query3 = '''SELECT ESTU_NSE_INDIVIDUAL, AVG(PUNT_GLOBAL) AS Promedio_Puntaje_Global
FROM data_icfes
GROUP BY ESTU_NSE_INDIVIDUAL
ORDER BY Promedio_Puntaje_Global DESC;
'''
result = pd.read_sql_query(query3, conn)
result

,ESTU_NSE_INDIVIDUAL,Promedio_Puntaje_Global
0,4.0,310.711786
1,3.0,269.237830
2,2.0,242.548306
3,NaN,241.788362
4,1.0,226.893151


In [12]:
# Top 5 municipios de residencia con mejores puntajes de matemáticas
query4 = '''SELECT ESTU_MCPIO_RESIDE, AVG(PUNT_MATEMATICAS) AS Promedio_Puntaje_Matematicas
FROM data_icfes
GROUP BY ESTU_MCPIO_RESIDE
ORDER BY Promedio_Puntaje_Matematicas DESC
LIMIT 5;
'''
result = pd.read_sql_query(query4, conn)
result

,ESTU_MCPIO_RESIDE,Promedio_Puntaje_Matematicas
0,MUTISCUA,65.562500
1,PAIPA,61.767782
2,PÁRAMO,60.836735
3,ENCISO,60.652174
4,NOBSA,60.625668


In [13]:
# Promedio de horas de trabajo semanal por género
query5 = '''SELECT 
    ESTU_GENERO, 
    AVG(
        CASE 
            WHEN ESTU_HORASSEMANATRABAJA = '0' THEN 0
            WHEN ESTU_HORASSEMANATRABAJA = 'Menos de 10 horas' THEN 5
            WHEN ESTU_HORASSEMANATRABAJA = 'Entre 11 y 20 horas' THEN 15
            WHEN ESTU_HORASSEMANATRABAJA = 'Entre 21 y 30 horas' THEN 25
            WHEN ESTU_HORASSEMANATRABAJA = 'Entre 31 y 40 horas' THEN 35
            WHEN ESTU_HORASSEMANATRABAJA = 'Más de 40 horas' THEN 45
            ELSE NULL
        END
    ) AS Promedio_Horas_Trabajo
FROM data_icfes
WHERE ESTU_HORASSEMANATRABAJA IS NOT NULL
GROUP BY ESTU_GENERO;
'''
result = pd.read_sql_query(query5, conn)
result

,ESTU_GENERO,Promedio_Horas_Trabajo
0,None,4.285714
1,F,2.657556
2,M,4.459518


In [14]:
# Relación entre acceso a internet en casa y puntajes de lectura critica
query6 = '''SELECT FAMI_TIENEINTERNET, AVG(PUNT_LECTURA_CRITICA) AS Promedio_Lectura_Critica
FROM data_icfes
GROUP BY FAMI_TIENEINTERNET;
'''
result = pd.read_sql_query(query6, conn)
result


,FAMI_TIENEINTERNET,Promedio_Lectura_Critica
0,None,49.745673
1,No,48.916023
2,Si,54.820713


In [15]:
# Distribución de estudiantes por tipo de colegio (Público/Privado)
query7 = '''SELECT 
    COLE_NATURALEZA, 
    COUNT(*) AS Total_Estudiantes,
    ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM data_icfes)), 2) AS Porcentaje_Estudiantes
FROM data_icfes
GROUP BY COLE_NATURALEZA
ORDER BY Porcentaje_Estudiantes DESC;
'''
result = pd.read_sql_query(query7, conn)
result

,COLE_NATURALEZA,Total_Estudiantes,Porcentaje_Estudiantes
0,OFICIAL,428403,77.73
1,NO OFICIAL,122746,22.27


In [16]:
# Comparación de mediana puntajes globales entre colegios bilingues y no bilingues
query8 = '''WITH OrderedScores AS (
    SELECT 
        COLE_BILINGUE, 
        PUNT_GLOBAL, 
        ROW_NUMBER() OVER (PARTITION BY COLE_BILINGUE ORDER BY PUNT_GLOBAL) AS RowNum,
        COUNT(*) OVER (PARTITION BY COLE_BILINGUE) AS Total
    FROM data_icfes
    WHERE COLE_BILINGUE IS NOT NULL AND COLE_BILINGUE != 'None'
)
SELECT 
    COLE_BILINGUE, 
    CASE 
        WHEN Total % 2 = 0 THEN 
            (SELECT AVG(PUNT_GLOBAL) 
             FROM OrderedScores AS Sub 
             WHERE Sub.COLE_BILINGUE = OrderedScores.COLE_BILINGUE 
             AND Sub.RowNum IN (Total / 2, Total / 2 + 1))
        ELSE 
            (SELECT PUNT_GLOBAL 
             FROM OrderedScores AS Sub 
             WHERE Sub.COLE_BILINGUE = OrderedScores.COLE_BILINGUE 
             AND Sub.RowNum = (Total + 1) / 2)
    END AS Mediana_Puntaje_Global
FROM OrderedScores
GROUP BY COLE_BILINGUE;

'''
result = pd.read_sql_query(query8, conn)
result

,COLE_BILINGUE,Mediana_Puntaje_Global
0,N,251.0
1,S,240.0


In [12]:
# Top 5 departamentos con mayor número de estudiantes presentando el exámen
query9 = '''SELECT ESTU_DEPTO_PRESENTACION, COUNT(*) AS Total_Estudiantes
FROM data_icfes
GROUP BY ESTU_DEPTO_PRESENTACION
ORDER BY Total_Estudiantes DESC
LIMIT 5;'''

result = pd.read_sql_query(query9, conn)
result


,ESTU_DEPTO_PRESENTACION,Total_Estudiantes
0,BOGOTÁ,78152
1,ANTIOQUIA,73834
2,VALLE,38837
3,CUNDINAMARCA,36720
4,ATLANTICO,32606


In [ ]:
# Puntaje global promedio de estudiantes privados de libertad
query10 = '''SELECT ESTU_PRIVADO_LIBERTAD, AVG(PUNT_GLOBAL) AS Promedio_Puntaje_Global
FROM data_icfes
GROUP BY ESTU_PRIVADO_LIBERTAD;
'''
result = pd.read_sql_query(query10, conn)
result

,ESTU_PRIVADO_LIBERTAD,Promedio_Puntaje_Global
0,N,252.607412
1,S,199.225806
